In [7]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from tqdm import tqdm
from tad_mctc.data.molecules import mols as samples
from dxtb import Calculator
from dxtb.config import ConfigCache

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}
cache_config = ConfigCache(enabled=False, density=True, fock=True)
opts = {"verbosity": 2}

# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
# opts = dict(opts, **{"scf_mode": "reconnect", "scp_mode": "potential"})
opts = dict(opts, **{"scf_mode": "implicit", "scp_mode": "charges"})
# opts = dict(opts, **{"scf_mode": "reconnect"})

dxtb.timer.reset()
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

print(f"Calc scf mode: {calc.opts.scf.scf_mode}")
print(f"Calc scp mode: {calc.opts.scf.scp_mode}")

charges = calc.get_charges(pos, charges, scf_charges_guess=None)
print(f"Charges.shape: {charges.shape}")
dxtb.timer.print(v=1)

Calc scf mode: 1
Calc scp mode: 1
SCF Energy  : -326.18651248195374 Hartree.
Total Energy: -322.95378788487295 Hartree.
Charges.shape: torch.Size([550])


# Reconnect vs implicit testing

### Energy and forces

In [ ]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from tqdm import tqdm
from tad_mctc.data.molecules import mols as samples
from dxtb import Calculator
from dxtb.config import ConfigCache

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}
cache_config = ConfigCache(enabled=False, density=True, fock=True)
opts = {"verbosity": 2}

# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
opts = dict(opts, **{"scf_mode": "implicit", "scp_mode": "charges"})

dxtb.timer.reset()
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

e_implicit = calc.get_energy(pos, charges)
scf_charges = calc.get_charges(pos, charges, scf_charges_guess=None)
dxtb.timer.print(v=1)

In [ ]:
opts = dict(opts, **{"scf_mode": "reconnect", "scp_mode": "charges"})
dxtb.timer.reset()
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

e_reconnect = calc.get_density(pos, charges, scf_charges_guess=scf_charges)
dxtb.timer.print(v=1)

AssertionError: Reconnect mode requires a charges guess.

In [4]:
f_implicit = torch.autograd.grad(e_implicit, pos, retain_graph=True)[0]
f_reconnect = torch.autograd.grad(e_reconnect, pos, retain_graph=True)[0]

print(f"e_implicit: {e_implicit}, e_reconnect: {e_reconnect}")
# print(f"f_implicit: {f_implicit}")
# print(f"f_reconnect: {f_reconnect}")
print(f"max diff: {torch.max(torch.abs(f_implicit - f_reconnect))}")

e_implicit: -322.9537656683717, e_reconnect: -322.95376567278356
max diff: 3.2826345703146353e-05


### Torch.no_grad()

In [5]:
# Molecule
sample = samples["vancoh2"]
numbers = sample["numbers"].to(dd["device"])
positions = sample["positions"].clone().to(**dd)
charges = torch.tensor(0.0, **dd)

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
opts = dict(opts, **{"scf_mode": "implicit", "scp_mode": "charges"})

dxtb.timer.reset()
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

with torch.no_grad():
    e_implicit = calc.get_energy(pos, charges)
    scf_charges = calc.get_charges(pos, charges, scf_charges_guess=None)
    dxtb.timer.print(v=1)

SCF Energy  : -326.18649026545251 Hartree.
Total Energy: -322.95376566837172 Hartree.
SCF Energy  : -326.18649026545251 Hartree.
Total Energy: -322.95376566837172 Hartree.


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Calculator                  0.007           0.68
Classicals                  0.021           1.92
 - DispersionD3             0.018          87.45
 - Halogen                  0.000           1.01
 - Repulsion                0.002          10.98
Integrals                   0.042           3.86
 - Overlap                  0.031          75.30
 - Core Hamiltonian         0.010          24.51
SCF                         1.003          93.07
 - Interaction Cache        0.003           0.28
 - Potential                0.966          96.24
 - Fock build               0.030           2.95
 - Diagonalize              0.750          74.72
 - Density                  0.075           7.52
 - Charges               

In [6]:
opts = dict(opts, **{"scf_mode": "reconnect", "scp_mode": "charges"})

pos = positions.clone().requires_grad_(True)
dxtb.timer.reset()

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)
calc.opts.cache = cache_config

e_reconnect = calc.get_energy(pos, charges, scf_charges_guess=scf_charges)
dxtb.timer.print(v=1)

SCF Energy  : -326.18649026986435 Hartree.
Total Energy: -322.95376567278356 Hartree.


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Calculator                  0.009           9.15
Classicals                  0.012          12.77
 - DispersionD3             0.010          85.31
 - Repulsion                0.002          13.19
 - Halogen                  0.000           0.90
Integrals                   0.028          30.06
 - Overlap                  0.021          75.83
 - Core Hamiltonian         0.007          24.03
SCF                         0.041          43.65
 - Interaction Cache        0.003           6.85
 - Potential                0.035          85.18
 - Fock build               0.002           3.83
 - Diagonalize              0.027          65.57
 - Density                  0.003           6.22
 - Charges                  0.003           7.52
------------------------------------------------
Sum          

In [7]:
# f_implicit = torch.autograd.grad(e_implicit, pos, retain_graph=True)[0]
f_reconnect = torch.autograd.grad(e_reconnect, pos, retain_graph=True)[0]

print(f"e_implicit: {e_implicit}, e_reconnect: {e_reconnect}")
# print(f"f_implicit: {f_implicit}")
# print(f"f_reconnect: {f_reconnect}")
print(f"max diff: {torch.max(torch.abs(f_implicit - f_reconnect))}")

e_implicit: -322.9537656683717, e_reconnect: -322.95376567278356
max diff: 2.1025343133196372e-09
